In [24]:
!pip install pillow cryptography



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
from cryptography.fernet import Fernet
from PIL import Image

## AES Encryption

In [26]:
def GenerateKey():
    MessageKey= Fernet.generate_key()
    with open("Secret.key", "wb") as FileKey:
        FileKey.write(MessageKey)

def LoadKey():
    return open("Secret.key", "rb").read()

def EncryptM(message):
    MessageKey= LoadKey()
    cipher = Fernet(MessageKey)
    EncryptedMsg = cipher.encrypt(message.encode())
    return EncryptedMsg

def DecryptM(EncryptedMsg):
    MessageKey= LoadKey()
    cipher = Fernet(MessageKey)
    DecryptedMsg = cipher.decrypt(EncryptedMsg).decode()
    return DecryptedMsg


## Steganography Module Using PIL
- Least Significant Bit (LSB) technique to embed the encrypted message into the pixels of an image is used.

In [27]:
def to_bin(data):
    return ''.join([format(byte, '08b') for byte in data])

# Embed encrypted message in the image
def EmbedData(image_path, data):
    img = Image.open(image_path)
    BinaryData = to_bin(data)
    BinaryLen = len(BinaryData)
    ImgData = img.getdata()
    NewImgData = []

    Index = 0
    for pixel in ImgData:
        if Index < BinaryLen:
            NewPix = list(pixel)
            for i in range(3):
                if Index < BinaryLen:
                    NewPix[i] = NewPix[i] & ~1 | int(BinaryData[Index])
                    Index += 1
            NewImgData.append(tuple(NewPix))
        else:
            NewImgData.append(pixel)
    img.putdata(NewImgData)
    img.save("Decrypted.png")
    print("Data embedded and saved as 'Decrypted.png'.")

def ExtraxtingData(image_path):
    img = Image.open(image_path)
    ImgData = img.getdata()
    BinaryData = ""

    for pixel in ImgData:
        for i in range(3):
            BinaryData += str(pixel[i] & 1)
    Bytes = [BinaryData[i:i+8] for i in range(0, len(BinaryData), 8)]
    data = bytearray([int(byte, 2) for byte in Bytes if int(byte, 2) != 0])
    return bytes(data)


- **Key Generation:** A key is generated for AES encryption and saved to secret.key. This step is only needed once.
- **Encryption:** The message is encrypted using AES and converted to binary for embedding.
- **Embedding:** The encrypted data is embedded into the image using LSB steganography.
- **Extraction and Decryption:** The encrypted data is extracted from the image, then decrypted to retrieve the original message.

In [29]:
def main():
    # GenerateKey() # run only for first time
    MessageKey= LoadKey()
    print(f"Encryption Key: {MessageKey}")
    SecretMsg = "I'm shitba!"
    EncryptedMsg = EncryptM(SecretMsg)
    print(f"Encrypted Message: {EncryptedMsg}")
    EmbedData("./Img2.png", EncryptedMsg)
    FinalData = ExtraxtingData("./Decrypted.png")
    DecryptedMsg = DecryptM(FinalData)
    print(f"Decrypted Message: {DecryptedMsg}")

if __name__ == "__main__":
    main()


Encryption Key: b'6GdtxL90zXkARm8mwoCAZfQMO3-jfcxRqFj1imVmexE='
Encrypted Message: b'gAAAAABnC6qs3I0BxBUaasRwcVmAoeVXIyuRNzKpAxk5MbdcAeYPp5inrJwTi7kJdsuKjVX8J34OqFxfmb79FlamnOvLaGnRQQ=='
Data embedded and saved as 'Decrypted.png'.
Decrypted Message: I'm shitba!
